In [1]:
import pandas as pd

In [2]:
# Data ingestion
names = [ 'buying','maint','doors','persons','lug_boot','safety','class']
data =  pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data', names=names)

In [3]:
# check for null
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   maint     1728 non-null   object
 2   doors     1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
 6   class     1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [4]:
# check values
for col in data:
    print(col, data[col].unique())

buying ['vhigh' 'high' 'med' 'low']
maint ['vhigh' 'high' 'med' 'low']
doors ['2' '3' '4' '5more']
persons ['2' '4' 'more']
lug_boot ['small' 'med' 'big']
safety ['low' 'med' 'high']
class ['unacc' 'acc' 'vgood' 'good']


In [13]:
ord_var = data[['buying', 'maint', 'lug_boot', 'safety', 'class']]
cat_var = data[['doors', 'persons']]

In [14]:
ord_var = ord_var.replace({'buying': {'low': 0, 'med': 1, 'high': 2, 'vhigh': 3},
                           'maint': {'low': 0, 'med': 1, 'high': 2, 'vhigh': 3},
                           'lug_boot': {'small': 0, 'med': 1, 'big': 2},
                           'safety': {'low': 0, 'med': 1, 'high': 2},
                           'class': {'unacc': 0, 'acc': 1, 'good': 2, 'vgood': 3}})
cat_var = pd.get_dummies(cat_var, drop_first = True)
df = pd.concat([ord_var, cat_var], axis=1)

In [15]:
y = df['buying']
x = df.drop('buying', axis = 1)

In [16]:
from sklearn.model_selection import (
    train_test_split,
    RandomizedSearchCV,
    StratifiedKFold,
)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=21)
params = [{"criterion": ("gini", "entropy"), "min_samples_leaf": (3, 5, 10, 20, 50)}]
rfc = RandomForestClassifier(random_state=21)
cla = RandomizedSearchCV(
    rfc,
    param_distributions=params,
    n_iter=10,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=21)
)
cla.fit(x_train, y_train)
y_pred = cla.predict(x_test)
metrics.accuracy_score(y_test, y_pred)

0.27167630057803466

In [20]:
# Maintenance = High,  Number of doors = 4, Lug Boot Size = Big, Safety = High Class Value = Good
cla.predict([[2, 2, 2, 2, 0, 1, 0, 0, 0],
             [2, 2, 2, 2, 0, 1, 0, 1, 0],
             [2, 2, 2, 2, 0, 1, 0, 0, 1]])

array([0, 0, 0], dtype=int64)

Based on the prediction, the price of the car is low for the different number of persons